In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x5761e000 @  0x7f995a3982a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                              ])
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Processing...
Done!


In [23]:
model = nn.Sequential(nn.Linear(784,128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,10),nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

images , labels = next(iter(trainloader))

images = images.flatten(start_dim=1)

logits = model(images)

loss = criterion(logits,labels)

print(loss)


tensor(2.3070, grad_fn=<NllLossBackward>)


In [4]:
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax()
)


In [5]:
print(criterion)

NLLLoss()


In [6]:
print(logits.shape)

torch.Size([64, 10])


In [7]:
print(images)
print(labels)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])
tensor([2, 3, 0, 3, 1, 4, 6, 2, 2, 4, 6, 9, 9, 7, 9, 6, 4, 2, 1, 8, 9, 5, 1, 0,
        3, 9, 1, 9, 5, 1, 6, 8, 5, 7, 9, 4, 3, 2, 9, 3, 9, 6, 7, 5, 6, 9, 7, 5,
        5, 3, 2, 6, 0, 2, 9, 8, 7, 4, 0, 4, 1, 7, 9, 4])


*AUTOGRAD PYTORCH*

In [18]:
x = torch.randn(2,2,requires_grad=True)
y=x**2
z=y.mean()
print(x)
print(y)
print(z)
print(x.grad)
print(y.grad)
print(z.grad)

tensor([[-0.2187,  0.4406],
        [ 2.8242,  0.4601]], requires_grad=True)
tensor([[0.0478, 0.1941],
        [7.9759, 0.2117]], grad_fn=<PowBackward0>)
tensor(2.1074, grad_fn=<MeanBackward1>)
None
None
None


In [19]:
z.backward()
print(x.grad)
print(x/2)

tensor([[-0.1093,  0.2203],
        [ 1.4121,  0.2300]])
tensor([[-0.1093,  0.2203],
        [ 1.4121,  0.2300]], grad_fn=<DivBackward0>)


NOW USING AUTOGRAD FOR NEURAL NETWORKS

In [24]:
print("weights before backward pass :\n",model[0].weight)
loss.backward()
print("weights after backward pass  :\n",model[0].weight.grad)

weights before backward pass :
 Parameter containing:
tensor([[-0.0051,  0.0137,  0.0291,  ..., -0.0220, -0.0119, -0.0147],
        [-0.0130, -0.0193, -0.0049,  ..., -0.0307,  0.0180, -0.0147],
        [-0.0154, -0.0270, -0.0231,  ...,  0.0320, -0.0047,  0.0043],
        ...,
        [ 0.0111, -0.0250,  0.0215,  ...,  0.0251, -0.0051,  0.0262],
        [ 0.0235, -0.0178,  0.0252,  ..., -0.0130, -0.0222,  0.0325],
        [ 0.0241, -0.0127, -0.0039,  ..., -0.0009,  0.0348,  0.0224]],
       requires_grad=True)
weights after backward pass  :
 tensor([[ 0.0017,  0.0017,  0.0017,  ...,  0.0017,  0.0017,  0.0017],
        [-0.0015, -0.0015, -0.0015,  ..., -0.0015, -0.0015, -0.0015],
        [ 0.0030,  0.0030,  0.0030,  ...,  0.0030,  0.0030,  0.0030],
        ...,
        [ 0.0002,  0.0002,  0.0002,  ...,  0.0002,  0.0002,  0.0002],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0001, -0.0001, -0.0001,  ..., -0.0001, -0.0001, -0.0001]])


In [0]:
from torch import optim

In [0]:
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)